# Downloading SRTM DEM from Google Earth Engine

## 1. Setting the scene  

### 1.1. Import required libraries

In [11]:
import ee                     # Earth Engine Python AP: provides access to Google's geospatial datasets and analysis capabilities
import geemap                 # simplifies interaction with Earth Engine and allows exporting of raster images
import geopandas as gpd       # used to handle shapefiles and other vector spatial data in Python
import numpy as np
import rasterio
import rasterio.plot
import matplotlib.pyplot as plt  # Library for creating visualisations

### 1.2.Define paths to datasets

In [12]:
m_root=r"C:/Users/VICTUS/Documents/MG(2)/programacion_sig/notebook/Workshop/PS/data"
aoi=m_root+r"/aoi.gpkg" # Path to the vector file that contains the AOI
dem_path=m_root+"/dem_aoi.tif" # Path where the downloaded DEM will be stored

## 2. Authentication and Connection to Google Earth Engine  

### 2.1: Authenticate with Google Earth Engine  

This step authorises access to your Google Earth Engine account from this environment.
A link will be generated; please sign in with your Google account and copy the verification code provided.

Note: This step is only required the first time you use Earth Engine in a new environment or after resetting credentials. Once authenticated, the credentials will be saved and reused automatically in subsequent sessions.

In [13]:
ee.Authenticate()

True

### 2.2. Initialise the Earth Engine Library  

Once authenticated, you must initialise the Earth Engine API to enable access to its functions in this notebook.

In [14]:
ee.Initialize()

## 3. Prepare and Download the SRTM DEM for Your Area of Interest  

### 3.1. Prepare and visualise the DEM  

We will load a shapefile representing the study area and convert its geometry into a format compatible with Earth Engine.  

We then use this geometry to clip the SRTM Digital Elevation Model (DEM), provided by the USGS at 30 m resolution, so that the dataset only covers the area of interest.

In [15]:
# Read the shapefile containing the extent of the study area
gdf_aoi = gpd.read_file(aoi)
print(gdf_aoi.crs)

#--- Convert the AOI geometry to Earth Engine format ---

aoi_geom = gdf_aoi.geometry[0]                    # Extract the polygon geometry
aoi_coords = aoi_geom.__geo_interface__       # Convert to GeoJSON format
aoi_ee = ee.Geometry(aoi_coords)              # Convert to Earth Engine Geometry

# --  Load the DEM and clip it to the AOI ---

# Use the SRTM DEM (30m resolution) provided by the USGS
dem = ee.Image("USGS/SRTMGL1_003")

# Clip the DEM to the area of interest
dem_clipped = dem.clip(aoi_ee)



EPSG:4326


## 4. Visualise the Clipped DEM on an Interactive Map  

This step uses geemap to display the DEM overlaid on an interactive map, with a colour palette representing elevation values.

In [16]:
# Create an interactive map centred on the centroid of the AOI
Map = geemap.Map(center=[aoi_geom.centroid.y, aoi_geom.centroid.x], zoom=13)

# Add the clipped DEM to the map with a custom colour palette
# Lower elevations are shown in red, and higher elevations in green
Map.addLayer(
    dem_clipped,
    {
        'min': 200,  # Minimum elevation for visualisation
        'max': 1000,  # Maximum elevation for visualisation
        'palette': ['red', 'orange', 'yellow', 'lightgreen', 'green']  # Colour scale
    },
    'SRTM DEM clipped'  # Layer name
)

# Display the map
Map

Map(center=[4.831080121156068, -73.16777407883282], controls=(WidgetControl(options=['position', 'transparent_…

### 🎯 Now it’s your turn  

Try modifying the parameters of the interactive map to explore a different area:  

- Replace the current shapefile with one representing another region of interest.  
- Adjust the zoom level and map centre accordingly.  
- Update the visualisation range (min and max) to better reflect the elevation characteristics of your new area.  
- Experiment with different colour palettes to emphasise various elevation gradients.

*Tip*: You can find alternative colour palettes on ColorBrewer or create your own.  

Once updated, re-run the map cell to check how the visualisation changes with your new parameters.

In [17]:
# Path to GeoPackage
jerico=m_root+r"/jerico.gpkg"


# Read the shapefile containing the extent of the study area
gdf_aoi = gpd.read_file(jerico)
gdf_aoi = gdf_aoi.to_crs("EPSG:4326")  # Ensure the CRS is WGS84
print(gdf_aoi.crs)

#--- Convert the AOI geometry to Earth Engine format ---

aoi_geom = gdf_aoi.geometry[0].simplify(0.001)                    # Extract the polygon geometry
aoi_coords = aoi_geom.__geo_interface__       # Convert to GeoJSON format
aoi_ee = ee.Geometry(aoi_coords)              # Convert to Earth Engine Geometry

# --  Load the DEM and clip it to the AOI ---

# Use the SRTM DEM (30m resolution) provided by the USGS
dem = ee.Image("USGS/SRTMGL1_003")

# Clip the DEM to the area of interest
dem_clipped = dem.clip(aoi_ee)

EPSG:4326


Map

In [18]:
# Create an interactive map centred on the centroid of the AOI
Map = geemap.Map(center=[aoi_geom.centroid.y, aoi_geom.centroid.x], zoom=10)

# Add the clipped DEM to the map with a custom colour palette
# Lower elevations are shown in red, and higher elevations in green
Map.addLayer(
    dem_clipped,
    {
        'min': 200,  # Minimum elevation for visualisation
        'max': 1000,  # Maximum elevation for visualisation
        'palette': ['red', 'orange', 'yellow', 'lightgreen', 'green']  # Colour scale
    },
    'SRTM DEM clipped'  # Layer name
)

# Display the map
Map

Map(center=[5.769830849334889, -75.76488244420928], controls=(WidgetControl(options=['position', 'transparent_…

## 5. Export the clipped DEM to a file on your local computer  

Once the SRTM DEM has been clipped to your area of interest and visually inspected, it can be exported to your local machine as a GeoTIFF file.  

The export function below uses the geemap.ee_export_image() utility, which handles the download of Earth Engine images.  

After running the cell, check your working directory for the output file.

In [19]:
# Export settings:
# - scale: 30 metres
# - region: defined by AOI
# - file_per_band=False: saves as a single-band GeoTIFF file

geemap.ee_export_image(
    dem_clipped,
    filename=dem_path,
    scale=30,
    region=aoi_ee,
    file_per_band=False
)

print(" Export complete ", dem_path)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\VICTUS\Documents\MG(2)\programacion_sig\notebook\Workshop\PS\data\dem_aoi.tif
 Export complete  C:/Users/VICTUS/Documents/MG(2)/programacion_sig/notebook/Workshop/PS/data/dem_aoi.tif
